# Canal bouillant 1D

In [ ]:
from trustutils.jupyter import run

run.introduction("Y.G.")
run.description("Comparaison a une solution analytique, voir STMF/LMEC/NT/15-044, paragraphe 6.2")
run.TRUST_parameters()

In [ ]:
from trustutils.jupyter import run

run.addCase("." ,"canal.data")
run.printCases()
run.runCases()

## Comparaison a la solution analytique

In [ ]:
from trustutils import plot

a = plot.Graph("h_m",nX=4,nY=2)
a.addSegment("canal_H_M_TRUST.son",label="trust")
a.addSegment("canal_H_M_ANALYTIQUE.son",label="analytique")

a.addPlot([0,1],'alpha')
a.addSegment("canal_ALPHA_TRUST.son",label="trust")
a.addSegment("canal_ALPHA_ANALYTIQUE.son",label="analytique")

a.addPlot([1,0],'temperature_l')
a.addSegment("canal_TEMPERATURE_L_TRUST.son")

a.addPlot([1,1],'temperature_g')
a.addSegment("canal_TEMPERATURE_G_TRUST.son")

a.addPlot([2,0],'pression')
a.addSegment("canal_PRESSION_TRUST.son")

a.addPlot([2,1],'RHO_M')
a.addSegment("canal_RHO_M_TRUST.son")

a.addPlot([3,0],'erreur_h')
a.addSegment("canal_ERREUR_H_TRUST.son")

a.addPlot([3,1],'vitesse_l')
a.addSegment("canal_VITESSE_L_TRUST.son",compo=1)
